### Download VIC Parl Ctte Submission PDFs and merge
* V0.1
* Jake Clarke, 24 July 2017

### settings

In [44]:
# path to submissions page
url = 'https://www.parliament.vic.gov.au/fireservicesbill/inquiries/article/2571'
# path to download url of the pdfs we find
path = 'https://www.parliament.vic.gov.au'
# where should I save these on disk?
outputpath = '_tmp_submissions'
mergepath = '_tmp_submissions/merge'

print(url)
print(path)
print(outputpath)
print(mergepath)

https://www.parliament.vic.gov.au/fireservicesbill/inquiries/article/2571
https://www.parliament.vic.gov.au
_tmp_submissions
_tmp_submissions/merge


### libraries

In [42]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import wget
import os
from os import listdir
from os.path import isfile, join
from datetime import *
from reportlab.pdfgen import canvas
from pdfrw import PdfReader, PdfWriter, IndirectPdfDict, PageMerge
from PyPDF2 import PdfFileWriter, PdfFileReader

### helper functions for getting submissions

In [18]:
# for an element in a list, find the url
def get_url(block, outputpath):
    link = block.find('a',{'class':'wf_file'})
    file_path = link['href']
    downloadurl = outputpath + file_path
    return downloadurl

# for a given url, download file
def get_file(url, outputpath):
    wget.download(url, out=outputpath)
    
# if download folder exists, empty it
# if it doesn't, make it
def download_folder(outputpath):
    if os.path.isdir(outputpath):
        for the_file in os.listdir(outputpath):
            file_path = os.path.join(outputpath, the_file)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
            except Exception as e:
                print(e)
    else:
        os.makedirs(outputpath)

### Get the page, download files

In [20]:
# get the page data
headers = {'User-Agent': UserAgent().chrome}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html5lib')

# make sure download directory exists and is empty
download_folder(outputpath)

# find first ordered list, loop through all <li>s and get url
submissionlist = soup.find('ol')
# for sub in submissionlist.findAll('li'):
#     data = get_url(sub, path)
#     get_file(data)

sub = submissionlist.find('li')
url = get_url(sub, path)
get_file(url, outputpath)

sub = sub.find_next('li')
url = get_url(sub, path)
get_file(url, outputpath)

sub = sub.find_next('li')
url = get_url(sub, path)
get_file(url, outputpath)

sub = sub.find_next('li')
url = get_url(sub, path)
get_file(url, outputpath)

sub = sub.find_next('li')
url = get_url(sub, path)
get_file(url, outputpath)

### Merge into one PDF

#### Helper functions

In [29]:
def get_files(outputpath):
    files = [f for f in listdir(outputpath) if isfile(join(outputpath, f))]
    return files

## function - make watermark pdf
## this is a pdf containing the name of a pdf, and the time we ran this script
## this way when we make one merged pdf, each page will be stamped with its original file
def make_watermark_pdf(fname,fwatermarkname):
    
    c = canvas.Canvas(fwatermarkname)
    c.setFont("Helvetica", 8)

    text = 'file: ' + str(fname)
    c.drawString(20,60,text)

    ts = datetime.now()
    ts_str = 'downloaded: ' + str(ts)
    c.drawString(20,50,ts_str)

    c.save()
    
### this function will take a pdf and merge it with a watermark pdf
### this makes a copy of each pdf, with the watermark
def merge_with_watermark(fname, foutputname, fwatermarkname):
    output = PdfFileWriter()

    ipdf = PdfFileReader(open(fname, 'rb'))
    wpdf = PdfFileReader(open(fwatermarkname, 'rb'))
    watermark = wpdf.getPage(0)

    for i in xrange(ipdf.getNumPages()):
        page = ipdf.getPage(i)
        page.mergePage(watermark)
        output.addPage(page)

    with open(foutputname, 'wb') as f:
        output.write(f)

In [30]:
# test get_files()
files = get_files(outputpath)
print(files[:10])

['S1-Anne_Laver.pdf', 'S2-CONFIDENTIAL.pdf', 'S3-Alister_Noonan.pdf', 'S4-Kenneth_Dwight.pdf', 'S5-Nicola_Stephens.pdf', 'watermark.pdf']


In [31]:
# test make_watermark_pdf()
make_watermark_pdf(
    fname = 'S1-Anne_Laver.pdf',
    fwatermarkname = '_tmp_submissions\watermark.pdf'
)

In [46]:
# test merge_with_watermark()
merge_with_watermark(
    fname = '_tmp_submissions\S1-Anne_Laver.pdf',
    foutputname = '_tmp_submissions\merge\S1-Anne_Laver.pdf',
    fwatermarkname = '_tmp_submissions\watermark.pdf'
)

In [50]:
# watermark pdf to use
fwatermarkname = mergepath + '/watermark/watermark.pdf'

# get file list
files = get_files(outputpath)

# make sure the subdir for merging is there
download_folder(mergepath)

#watermark dir
download_folder(mergepath + '/' + 'watermark')

for file in files:

    fname = outputpath + '/' + file
    foutputname = mergepath + '/' + file
    
    # make watermark file containing date stamp and file name
    
    make_watermark_pdf(
        fname,
        fwatermarkname
    )
    
    # merge with watermark file
    
    merge_with_watermark(
        fname,
        foutputname,
        fwatermarkname
    )

### merge into one PDF

In [183]:
# merge files from merge subdir

writer = PdfWriter()

for file in files:
    fname = mergepath + '/' + file
    writer.addpages(PdfReader(fname).pages)

writer.write(outputpath + '/' + 'S000_merge.pdf')